In [ ]:
import numpy as np
import cv2
import os

 
images_folder = './N2-embryo-10um-SRx5-crop-images-folders'
seg_folder = './N2-embryo-10um-SRx5-crop-seg-folders'

sub_folders = os.listdir(images_folder)

print(sub_folders)

crop_imags_correct_angle_folder = "N2-embryo-10um-SRx5-crop-images-correct-angle-folders"
crop_seg_correct_angle_folder = "N2-embryo-10um-SRx5-crop-seg-correct-angle-folders"


try:
    os.makedirs(crop_imags_correct_angle_folder)
    print(f"Folder '{crop_imags_correct_angle_folder}' created successfully.")
except FileExistsError:
    print(f"Folder '{crop_imags_correct_angle_folder}' already exists.")   


try:
    os.makedirs(crop_seg_correct_angle_folder)
    print(f"Folder '{crop_seg_correct_angle_folder}' created successfully.")
except FileExistsError:
    print(f"Folder '{crop_seg_correct_angle_folder}' already exists.")   


for sub_folder in sub_folders:

    sub_crop_imags_folder = crop_imags_correct_angle_folder + '/' + sub_folder
    sub_crop_segs_folder = crop_seg_correct_angle_folder + '/' + sub_folder
    try:
        os.makedirs(sub_crop_imags_folder)
        os.makedirs(sub_crop_segs_folder)
    except FileExistsError:
        pass

In [ ]:
import math


for sub_folder in sub_folders:

    images_list_dir = images_folder+'/'+sub_folder
    seg_images_list_dir = seg_folder+'/'+sub_folder

    images_list = os.listdir(images_list_dir)
    seg_images_list = os.listdir(seg_images_list_dir)


    
    
    


    
    
    print(sub_folder)

    for image_name in images_list:
        
        if image_name.endswith('.png'):

            
            origin_image = cv2.imread(images_list_dir + '/' + image_name)
            mask = cv2.imread(seg_images_list_dir + '/' + image_name, cv2.IMREAD_GRAYSCALE)

            h,w,c = origin_image.shape


            
            contours = cv2.findContours(
                mask.astype("uint8"), cv2.RETR_CCOMP,
                cv2.CHAIN_APPROX_NONE)[-2]


            mu=cv2.moments(contours[0],False)
            center_x = int(mu['m10'] / (mu['m00']+0.00000001))
            center_y = int(mu['m01'] / (mu['m00']+0.00000001))
            center = (center_x,center_y) 

            cut_scale_based_center_point = 0.3

            top = int(360 - 360*cut_scale_based_center_point)
            bottom = int(360 + 360*cut_scale_based_center_point)
            mask[0:top,:] = 0
            mask[bottom+1:720,:] = 0

            left_top_min = 1000
            left_bottom_min = 1000

            right_top_max = 0
            right_bottom_max = 0

            for point in contours[0]:
                x,y = point[0]
                if y == top:
                    if x > right_top_max:
                        right_top_max = x
                    if x < left_top_min:
                        left_top_min = x
                if y == bottom:
                    if x > right_bottom_max:
                        right_bottom_max = x
                    if x < left_bottom_min:
                        left_bottom_min = x

            left_top = (left_top_min,top)
            right_top = (right_top_max,top)

            left_bottom = (left_bottom_min,bottom)
            right_bottom = (right_bottom_max,bottom)

            

            
            
            

            
            dX = right_bottom_max - right_top_max
            dY = bottom - top       
            d = math.sqrt(dX**2 + dY**2)
            theta = math.atan2(dY, dX)      
            right_angle = math.degrees(theta)


            dX = left_bottom_min - left_top_min
            dY = bottom - top
            d = math.sqrt(dX**2 + dY**2)
            theta = math.atan2(dY, dX)      
            left_angle = math.degrees(theta)                


            diff_angle = (right_angle+left_angle)/2 - 90



            img2 = cv2.imread(images_list_dir + '/' + image_name)
            M = cv2.getRotationMatrix2D(center, diff_angle, 1.0)
            img2 = cv2.warpAffine(img2, M, (w, h))
            cv2.imwrite(crop_imags_correct_angle_folder +'/'+ sub_folder+ '/'  + image_name,img2)

            mask2 = cv2.imread(seg_images_list_dir + '/' + image_name)
            mask2 = cv2.warpAffine(mask2, M, (w, h))
            cv2.imwrite(crop_seg_correct_angle_folder   +'/'+ sub_folder+ '/' + image_name,mask2)

        

In [ ]:
    for image_name in images_list:
        
            
        origin_image = cv2.imread(images_list_dir + '/' + image_name)
        mask = cv2.imread(seg_images_list_dir + '/' + image_name, cv2.IMREAD_GRAYSCALE)

        h,w,c = origin_image.shape


        
        contours = cv2.findContours(
            mask.astype("uint8"), cv2.RETR_CCOMP,
            cv2.CHAIN_APPROX_NONE)[-2]


        mu=cv2.moments(contours[0],False)
        center_x = int(mu['m10'] / (mu['m00']+0.00000001))
        center_y = int(mu['m01'] / (mu['m00']+0.00000001))
        center = (center_x,center_y) 

        cut_scale_based_center_point = 0.3

        top = int(360 - 360*cut_scale_based_center_point)
        bottom = int(360 + 360*cut_scale_based_center_point)
        mask[0:top,:] = 0
        mask[bottom+1:720,:] = 0

        left_top_min = 1000
        left_bottom_min = 1000

        right_top_max = 0
        right_bottom_max = 0

        for point in contours[0]:
            x,y = point[0]
            if y == top:
                if x > right_top_max:
                    right_top_max = x
                if x < left_top_min:
                    left_top_min = x
            if y == bottom:
                if x > right_bottom_max:
                    right_bottom_max = x
                if x < left_bottom_min:
                    left_bottom_min = x

        left_top = (left_top_min,top)
        right_top = (right_top_max,top)

        left_bottom = (left_bottom_min,bottom)
        right_bottom = (right_bottom_max,bottom)

        

        
        
        

        
        dX = right_bottom_max - right_top_max
        dY = bottom - top       
        d = math.sqrt(dX**2 + dY**2)
        theta = math.atan2(dY, dX)      
        right_angle = math.degrees(theta)


        dX = left_bottom_min - left_top_min
        dY = bottom - top
        d = math.sqrt(dX**2 + dY**2)
        theta = math.atan2(dY, dX)      
        left_angle = math.degrees(theta)                


        diff_angle = (right_angle+left_angle)/2 - 90



        img2 = cv2.imread(images_list_dir + '/' + image_name)
        M = cv2.getRotationMatrix2D(center, diff_angle, 1.0)
        img2 = cv2.warpAffine(img2, M, (w, h))
        cv2.imwrite(crop_imags_correct_angle_folder +'/'+ sub_folder+ '/'  + image_name,img2)

        mask2 = cv2.imread(seg_images_list_dir + '/' + image_name)
        mask2 = cv2.warpAffine(mask2, M, (w, h))
        cv2.imwrite(crop_seg_correct_angle_folder   +'/'+ sub_folder+ '/' + image_name,mask2)
